Да бисте покренули следеће свеске, ако то већ нисте урадили, потребно је да распоредите модел који користи `text-embedding-ada-002` као основни модел и подесите име распореда унутар .env фајла као `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Затим ћемо учитати индекс угњеждавања у Pandas Dataframe. Индекс угњеждавања је сачуван у JSON фајлу под називом `embedding_index_3m.json`. Индекс угњеждавања садржи угњеждавања за сваки од YouTube транскрипата до краја октобра 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Затим ћемо направити функцију под називом `get_videos` која ће претраживати Embedding Index на основу упита. Функција ће вратити 5 видеа који су најсличнији упиту. Функција ради на следећи начин:

1. Прво се прави копија Embedding Index-а.
2. Затим се рачуна Embedding за упит помоћу OpenAI Embedding API-ја.
3. Онда се у Embedding Index-у додаје нова колона под називом `similarity`. Колона `similarity` садржи косинусну сличност између Embedding-а упита и Embedding-а за сваки видео сегмент.
4. Следеће, Embedding Index се филтрира по колони `similarity`. Embedding Index се филтрира тако да укључује само видеа која имају косинусну сличност већу или једнаку 0.75.
5. На крају, Embedding Index се сортира по колони `similarity` и враћа се 5 најсличнијих видеа.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ова функција је веома једноставна, она само исписује резултате претраге.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Прво се Ембединг индекс учитава у Pandas Dataframe.
2. Затим се од корисника тражи да унесе упит.
3. После тога се позива функција `get_videos` да би се претражио Ембединг индекс на основу упита.
4. На крају се позива функција `display_results` да би се кориснику приказали резултати.
5. Кориснику се затим поново тражи да унесе нови упит. Овај процес се понавља док корисник не унесе `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.sr.png)

Бићете упитани да унесете упит. Унесите упит и притисните ентер. Апликација ће вам приказати листу видеа који су релевантни за ваш упит. Апликација ће вам такође дати линк до дела видеа где се налази одговор на ваше питање.

Ево неких упита које можете пробати:

- Шта је Azure Machine Learning?
- Како функционишу конволуционе неуронске мреже?
- Шта је неуронска мрежа?
- Могу ли да користим Jupyter Notebooks са Azure Machine Learning?
- Шта је ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
